### This code works fine but the GUI library has a glitch that it kills kernel second time you run. 
### That's why we run it as a subprocess 

In [1]:
import numpy as np
import open3d as o3d
import PySimpleGUI as sg
import matplotlib.pyplot as plt
import copy
from PIL import Image
from io import BytesIO


sample_ply_data = o3d.data.DemoCropPointCloud()
tst_pointCloud = o3d.io.read_point_cloud(sample_ply_data.point_cloud_path)
#o3d.visualization.draw_geometries([tst_pointCloud])

def array_to_data(array):
    im = Image.fromarray(array)
    output_buffer = BytesIO()
    im.save(output_buffer, format="PNG")
    data = output_buffer.getvalue()
    return data

def fetch_cloud_image(pointCloud, RX=0, RY=0, RZ=0):
    tmp_Rot = pointCloud.get_rotation_matrix_from_xyz((np.degrees(RX), np.degrees(RY), np.degrees((RZ))))
    tmp_cloud = copy.deepcopy(pointCloud)  #To avoid overwriting the original point cloud
    tmp_cloud.rotate(tmp_Rot, center=(0, 0, 0))
    vis = o3d.visualization.Visualizer() 
    vis.create_window(visible=False, width=640, height=480) 
    vis.add_geometry(tmp_cloud) 
    vis.poll_events() 
    vis.update_renderer() 
    color = vis.capture_screen_float_buffer(True) 
    vis.destroy_window() 
    #color = np.asarray(color)
    color = (255.0 * np.asarray(color)).astype(np.uint8)
    color = array_to_data(color) #Format according to the GUI requirements
    return color



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
def Cluster_selection_gui(clouds, RX = 0, RY = 0, RZ = 0):
    
    dat=[]  #to hold cluster's images
    selected_PC = [] #will hold index values of selected point clouds.
    pcd_combined = o3d.geometry.PointCloud()
    
    for current_cloud in clouds:
            color = fetch_cloud_image(current_cloud, RX, RY, RZ)  #different point clouds to be viewed
            dat.append(color)


    file_list_column = [[sg.Text("Select all valid Clusters"),]]
    for i in range (len(dat)):
        #print(i)
        file_list_column.append( [sg.Button(button_text = "",enable_events = True, 
                                     size = (16, 4),key = 'but'+str(i), image_source=dat[i], 
                                     image_size = (None, None), image_subsample = 5, )] )

    buttons_layout = [
        [sg.Text("Buttons",size=(40, 1))],
        [ sg.Button(button_text = "Cancel",enable_events = True, 
                                     size = (8, 2),key = 'Cancel', image_source=None, 
                                     image_size = (None, None), image_subsample = None, ) ,
         sg.Button(button_text = "OK",enable_events = True, 
                                     size = (8, 2),key = 'OK', image_source=None, 
                                     image_size = (None, None), image_subsample = None, ) ] #to put buttons in single row
                     ]
    image_viewer_column = [
        [sg.Text("Preview")],
        [sg.Text(size=(40, 1), key="-TOUT-")],
        [sg.Image(key="img",source=dat[0])], [sg.HSeparator(pad=(0,10))], [sg.Column(buttons_layout)]  ]


    layout = [ [ sg.Column(file_list_column,size = (None, None), 
                           scrollable=True, vertical_scroll_only=True,expand_y=True ), 
                sg.VSeparator(), 
                sg.Column(image_viewer_column), ] 
             ]

    window = sg.Window("Image Viewer", layout)

    while True: # Run the Event Loop
        event, values = window.read()
        try:
            event_name = event.rstrip('0123456789')
            event_idx = int(event[len(event_name):])
            #print(event_name, event_idx)
        except:
            pass

        if event == "Cancel" or event == sg.WIN_CLOSED:
            selected_PC = []
            break

        elif event_name == "but":
            try:
                window["img"].update(source=dat[event_idx])
            except:
                print("Error")
                pass
            #if event_idx == 0:

            if (event_idx in selected_PC) == True:
                available_index = selected_PC.index(event_idx)
                del_item = selected_PC.pop(available_index)
                window["but"+str(event_idx)].update(text = "", image_data=dat[event_idx], image_subsample = 5)
                selected_PC = sorted(selected_PC)
                #print(selected_PC)
            else:
                selected_PC.append(event_idx)
                window["but"+str(event_idx)].update(text = "SELECTED",  image_subsample = 5)
                selected_PC = sorted(selected_PC)
                #print(selected_PC)

        elif event == "OK":
            print("The current selection includes:",selected_PC)
            break

    window.close()

    for k in selected_PC:
        pcd_combined += clouds[k]  #combine selected pointclouds 

    return pcd_combined




In [ ]:
new_one = Cluster_selection_gui([tst_pointCloud,tst_pointCloud], RX = -45, RY = 0, RZ = 0)